## ПЗ 4

Построим тематические модели для корпуса новостей Lenta.ru.

In [ ]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [ ]:
data = [(record.title, record.topic, record.text, record.tags) for record in records]

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

<ipython-input-3-5b7176a9d163>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [ ]:
data = pd.DataFrame(data, columns=['title','topic','text','tags'])

In [ ]:
data.sample(3)

,title,topic,text,tags
23430,Евросоюз рассказал о судьбе России в «Большой восьмерке»,Мир,"Евросоюз не желает видеть Россию в формате «Большой восьмерки». Об этом заявил председатель Совета ЕС Дональд Туск, передает Deutsche Welle. По его словам, Москва сама не заинтересована в возвращении в формат G8, а семь — хорошее число. «Меня беспокоит, что основанный на правилах миропорядок ставится под вопрос, как это ни странно, его важнейшим архитектором и гарантом — Соединенными Штатами», — отметил он. Слова президента США Дональда Трампа о необходимости вернуть Россию в G8 раскритиковали также в МИД Канады, передавал CBC News. По словам спикера ведомства, позиция страны по этому вопросу не изменилась: Оттава считает впредь недопустимым присутствие президента России Владимира Путина на саммите. Ранее 8 июня американский лидер заявил, что Россия должна вернуться в клуб и присутствовать за столом переговоров. Позицию Трампа поддержал премьер-министр Италии Джузеппе Конте. 4 июня пресс-секретарь президента России Дмитрий Песков сообщил журналистам, что G8 теряет свою актуальность для российской стороны. По его словам, для Москвы более актуальны такие форматы, как «Большая двадцатка», в котором Россия принимает активное участие. Согласится ли Кремль присоединиться к «восьмерке» в случае соответствующего предложения, Песков не уточнил. Формат G8 прекратил свое существование в 2014 году после присоединения Крыма к России. Тогда лидеры ведущих мировых держав отказались участвовать в саммите «восьмерки», который Россия как председатель должна была проводить в Сочи. Вместо этого в Брюсселе прошла встреча «семерки» — G7 (Франция, Великобритания, США, Германия, Япония, Италия и Канада). В саммите также принимают участие представители Евросоюза (председатели Совета ЕС и Еврокомиссии).",Политика
255457,Денису Лебедеву вернули титул чемпиона мира по боксу,Спорт,"Российскому боксеру Денису Лебедеву вернули титул чемпиона мира по версии WBA (Всемирная боксерская ассоциация) в весе до 90,7 килограмма, которого он лишился после поражения в бою с Гильермо Джонсом из Панамы. Джонс, проваливший допинг-тест после этого поединка, признан виновным в применении запрещенных препаратов. Об этом сообщает интернет-издание Fightnews. Лишить Джонса титула постановил комитет WBA по чемпионским боям. По сведениям Fightnews, анализ контрольной допинг-пробы не проводился. Ранее Джонс утверждал, что контрольная проба была вскрыта и наличие в организме боксера допинга не подтвердилось. Будут ли к Джонсу применены какие-то дополнительные санкции (например, штраф или дисквалификация), не уточняется. По данным издания, WBA назначила бой-реванш между российским и панамским боксером. Если представители спортсменов не договорятся, право организации этого поединка будет выставлено на торги. Поединок Лебедев — Джонс состоялся 17 мая 2013 года в Москве. В этом бою Джонс победил техническим нокаутом в 11-м раунде. Благодаря победе над Лебедевым панамский боксер вернул чемпионский титул, которого его ранее лишили из-за того, что бой Джонса с россиянином несколько раз срывался (поединок с Лебедевым панамец проводил в статусе «временно недействующего» чемпиона). СМИ сообщали, что по окончании боя Джонс отказывался пройти допинг-тест и согласился только после долгих убеждений. Анализ пробы, проведенный в лаборатории Российского антидопингового агентства (РУСАДА), показал, что в организме спортсмена содержался запрещенный фуросемид. Этот мочегонный препарат сам по себе не помогает улучшить спортивные результаты, но может использоваться либо для сгонки веса, либо как маскирующий агент — чтобы скрыть следы применения стероидов. Узнав о результатах анализа, Джонс заявил, что не употреблял запрещенные вещества, а допинг-скандал назвал «оправданием для русских».",Бокс и ММА
596069,Шведская легкоатлетка побила мировой рекорд в прыжках в высоту,Спорт,"Шведская легкоатлетка Кайса Бергквист побила мировой рекорд по прыжкам в высоту в закрытых помещениях на соревнованиях в немецком Арнштадте. 

In [ ]:
len(data)

739350

In [ ]:
data.topic.value_counts().head(15)

Россия               160518
Мир                  136680
Экономика            79538 
Спорт                64421 
Культура             53803 
Бывший СССР          53402 
Наука и техника      53136 
Интернет и СМИ       44675 
Из жизни             27611 
Дом                  21734 
Силовые структуры    19596 
Ценности             7766  
Бизнес               7399  
Путешествия          6408  
69-я параллель       1268  
Name: topic, dtype: int64

Для ускорения вычислений оставим тексты из 5 тем:

In [ ]:
topics = ['Бывший СССР', 'Мир', 'Интернет и СМИ', 'Бизнес', 
          'Дом']


data = data[data.topic.isin(topics)]
data.dropna(inplace=True)
len(data)

263890

### Предобработка


Все модели, с которыми мы будем работать далее, работают с предобработанными текстами, поэтому сперва проведем стандартную предобработку текста:

In [ ]:
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [ ]:
data.text.iloc[1]

'Хакерская группировка Anonymous опубликовала новые документы о деятельности британского аналитического центра Integrity Initiative. Из материалов следует, что центр получает финансирование от некого Института государственного управления под руководством Кристофера Доннелли. Хакеры выложили в открытый доступ его паспорт, резюме, а также сведения о его связях с Министерством обороны Великобритании. Согласно документам, в 2014 году Доннелли выдвинул ряд предложений британским властям в связи с ситуацией в Крыму. В частности, он планировал заминировать Севастопольскую бухту, окружить полуостров войсками, а также уничтожить оставшиеся в Крыму самолеты «в знак серьезности намерений». После публикации заметки часть документов, на которые члены группировки ссылаются в своем расследовании, была удалена. Также хакеры утверждают, что Доннелли был инициатором расследования «российского вмешательства» в референдум о независимости в Каталонии. Он пригласил членов испанского отделения Integrity Init

Ниже мы оставим только случайные 10,000 текстов из нашей коллекции, чтобы ускорить вычисления:

In [ ]:
data = data.sample(10000)
data.topic.value_counts()

Мир               5251
Бывший СССР       1964
Интернет и СМИ    1687
Дом               819 
Бизнес            279 
Name: topic, dtype: int64

In [ ]:
preprocessed_text = list(tqdm(map(preprocess, data['text']), total=len(data)))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
data['text'] = preprocessed_text
data.sample(5)

,title,topic,text,tags
398591,ВС США получили нового начальника Центрального командования,Мир,"[сенат, утвердить, генерал, джеймс, мэттис, пост, глава, центральный, командование, вооружённый, сила, сша, centcom, сообщать, агентство, associated, press, мэттис, предстоять, руководить, военный, операция, ближний, восток, афганистан, ирак, джеймс, мэттис, стать, преемник, генерал, дэвид, петрэуса, который, свой, очередь, занять, должность, командующий, коалиционный, сила, афганистан, сменить, это, пост, генерал, стэнли, маккристал, маккристал, вынудить, уйти, отставка, публикация, скандальный, материал, немой, журнал, rolling, stone, июнь, год, мэттис, участвовать, военный, операция, ирак, афганистан, протяжение, несколько, год, главнокомандующий, объединить, сила, ирак, генерал, известный, эксцентричный, высказывание, различный, тема, частность, год, мэттис, служить, афганистан, грозить, дисциплинарный, взыскание, слово, стрелять, человек, который, бить, женщина, отказ, носить, паранджа, удовольствие, начальство, мэттис, ...]",Все
400676,К дочери Юлии Тимошенко пришел УБЭП,Бывший СССР,"[бывший, министр, внутренний, дело, украина, юрий, луценко, интервью, газета, киевск, рассказать, новый, руководство, милиция, заниматься, чистка, руководитель, политический, мотив, разгон, митинг, сбор, компромат, оппозиция, качество, пример, рассказать, история, прошлый, неделя, сидеть, тимошенко, пить, кофе, звонить, дочь, печерск, небольшой, кафешка, сотрудник, управление, борьба, экономический, преступление, устроить, обыск, полный, выемка, весь, документ, слово, луценко, выяснить, свой, канал, операция, проводиться, личный, контроль, начальник, столичный, милиция, бред, резюмировать, бывший, министр, закончиться, обыск, луценко, рассказывать, стать, ранее, сообщаться, евгений, тимошенко, супруг, шон, карра, год, франшиза, открыть, центр, киев, итальянский, ресторан, ciro, pomodoro, площадь, сто, квадратный, метр, кроме, родный, днепропетровск, тимошенко, младший, журналист, наслать, вареничный, кафе, ...]",Все
358691,В Измайловском парке снесли самострой,Дом,"[требование, прокуратура, измайловский, парка, москва, снести, незаконно, возвести, строение, это, говориться, сообщение, столичный, прокуратура, нарушение, природоохранный, законодательство, особо, охранять, природный, территория, природный, исторический, парк, измайлово, выявить, ход, проверка, который, провести, межрайонный, природоохранный, прокуратура, прокурор, установить, граница, парк, предприниматель, милехина, самовольно, занять, земельный, участок, площадь, квадратный, метр, необходимый, разрешительный, документ, возвести, немой, ряд, строение, предприниматель, неоднократно, привлекаться, административный, ответственность, нарушение, земельный, градостроительный, законодательство, однако, самострой, снести, итог, прокуратура, направить, измайловский, районный, суд, москва, иск, требование, освободить, участок, ликвидировать, незаконный, постройка, суд, требование, удовлетворить, настоящий, момент, всё, самовольный, строение, незаконно, занятой, участок, снести]",Все
394697,Сообщение о бомбе на борту тайского лайнера оказалось ложным,Мир,"[спецслужба, обнаружить, взрывчатый, вещество, борт, самолёт, авиакомпания, thai, airways, туалетный, комната, который, ранее, найти, записка, угроза, взрыв, это, сообщать, associated, press, сапёр, исследовать, самолёт, изнутри, досмотреть, багаж, пассажир, никакой, признак, бомба, взрывчатый, вещество, обнаружить, всё, вывести, самолёт, сразу, посадка, допросить, затем, отпустить, задержать, результат, инцидент, похоже, это, сделать, сумасшедший, заявить, президент, авиакомпания, thai, airways, пивасвастить, амрананд, pivasvasti, amranand, телефон, очень, жаль, прийтись, доставить, беспокойство, пассажир, сообщение, борт, самолёт, лететь, лос, анджелес, бангкок, возможно, заложить, бомба, поступить, примерно, калифорнийский, время, утро, москва, записка, который, найти, зеркало, туалет, сообщаться, самолёт, заминировать, текст, написать, анг

## Модель LDA


Mодель, которую мы рассмотрим, LDA - латентное размещение Дирихле. Воспользуемся реализацией из библиотеки sklearn.

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


stopwords_list = stopwords.words('russian')
n_features = 1000
n_components = 15
n_top_words = 10

new_corp = []

for sentence in preprocessed_text:
    sentence = ' '.join(sentence)
    new_corp.append(sentence)
    
corpus = new_corp

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, max_features=n_features, stop_words=stopwords_list, ngram_range=(1, 2))
tfidf = tfidf_vectorizer.fit_transform(corpus)

lda = LatentDirichletAllocation(n_components=n_components, random_state=1).fit(tfidf)


In [ ]:
len(lda.components_), len(lda.components_[0])

(15, 1000)

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()
word_dict = {}
for topic_idx, topic in enumerate(lda.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features = [feature_names[i] for i in top_features_ind]
    word_dict[topic_idx] = top_features
    

pd.DataFrame(word_dict)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,сша,украина,выборы,грузия,человек,язык,иран,суд,процент,израильский,компания,сирия,корея,пользователь,самолёт
1,президент,украинский,партия,россия,полиция,русский,интернет,год,год,израиль,год,сирийский,кндр,ролик,борт
2,обама,россия,парламент,грузинский,произойти,латвия,сайт,дело,рубль,палестинский,реклама,ирак,северный,фотография,пассажир
3,президент сша,президент,министр,российский,погибнуть,статус,пользователь,который,миллион,аль,интернет,турция,ядерный,видео,экипаж
4,трамп,это,процент,военный,взрыв,государственный,иранский,это,метр,боевик,сайт,оон,ракета,twitter,аэропорт
5,буш,киев,премьер,сша,пострадать,владимир,доступ,обвинение,строительство,палестинец,процент,турецкий,южный,опубликовать,судно
6,год,страна,голос,страна,город,белорусский,программа,бывший,доллар,военный,новый,повстанец,япония,сеть,полёт
7,американский,год,голосование,министр,результат,референдум,ядерный,свой,тысяча,ирак,который,боевик,санкция,facebook,находиться
8,свой,который,премьер министр,нато,сообщать,афганистан,сеть,задержать,квадратный,хамас,пользователь,военный,оружие,девушка,авиакомпания
9,штат,глава,кандидат,мид,местный,конфликт,компания,тюрьма,квадратный метр,сектор,миллион,страна,китай,который,корабль


In [ ]:
lda.perplexity(tfidf)

1882.8922635459753